## Python loading

In [194]:
import sys

import pandas as pd
from pandas import DataFrame, Series

import numpy as np

import matplotlib.pyplot as plt
import matplotlib

#import xgboost as xgb

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier 

%matplotlib inline

In [195]:
train = pd.read_csv("../../data/train.csv", index_col=0)
test = pd.read_csv("../../data/test.csv", index_col=0)

train.head(2)

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby


In [196]:
train[train.AnimalType == "Dog"].Name.value_counts()

Bella          113
Max            112
Charlie         89
Daisy           86
Buddy           78
Rocky           77
Lucy            76
Princess        72
Lola            53
Chico           53
Jack            51
Sadie           51
Luna            50
Coco            48
Lucky           45
Shadow          44
Lady            44
Duke            44
Blue            43
Maggie          41
Molly           40
Toby            39
Cookie          38
Bear            37
Sasha           37
Rosie           37
Diamond         36
Roxy            35
Buster          34
Ginger          33
              ... 
Mr Bentley       1
Truly            1
Normal           1
Hatch            1
Teah             1
Camillo          1
Ms. Scott        1
Juni             1
Marcos           1
Rojo             1
Luna Bella       1
Stefani          1
Zu Zu            1
Sheri            1
Julius           1
Copperfield      1
Mozzy            1
Indio            1
Mumble           1
Neva             1
Korg             1
Emmie Lou   

## Data fitting

In [197]:
train.SexuponOutcome.value_counts()

Neutered Male    9779
Spayed Female    8820
Intact Male      3525
Intact Female    3511
Unknown          1093
Name: SexuponOutcome, dtype: int64

In [198]:
train = pd.read_csv("../../data/train.csv", index_col=0)
test = pd.read_csv("../../data/test.csv", index_col=0)

def replaceing(dataset):
    opt = dataset.copy()

    opt["Intact"] = opt.SexuponOutcome.replace("Intact Male", "Yes")
    opt.Intact = opt.Intact.replace("Neutered Male", "No")
    opt.Intact = opt.Intact.replace("Spayed Female", "No")
    opt.Intact = opt.Intact.replace("Intact Female", "Yes")
    opt.Intact = opt.Intact.replace("Unknown", np.nan)

    opt.SexuponOutcome = opt.SexuponOutcome.replace("Neutered Male", "Male")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Male", "Male")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Spayed Female", "Female")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Female", "Female")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Unknown", np.nan)

    return opt

train = replaceing(train)
test = replaceing(test)


## Data categorising

In [199]:
train.Intact = train.Intact.astype("category")
train["IntactValue"] = train.Intact.cat.codes
test.Intact = test.Intact.astype("category", train.Intact.cat.categories.values.tolist())
test["IntactValue"] = test.Intact.cat.codes

train.SexuponOutcome = train.SexuponOutcome.astype("category")
train["SexuponOutcomeValue"] = train.SexuponOutcome.cat.codes
test.SexuponOutcome = test.SexuponOutcome.astype("category", train.SexuponOutcome.cat.categories.values.tolist())
test["SexuponOutcomeValue"] = test.SexuponOutcome.cat.codes

train.OutcomeType = train.OutcomeType.astype("category")

In [200]:
train.Color = train.Color.apply(lambda x: x.split("/")[0])
test.Color = test.Color.apply(lambda x: x.split("/")[0])
len(train.Color.value_counts())

57

In [201]:
train.Color = train.Color.astype("category")
train["ColorValue"] = train.Color.cat.codes

test.Color = test.Color.astype("category", categories=train.Color.cat.categories.values.tolist())
test["ColorValue"] = test.Color.cat.codes

In [202]:
test[test.Color == "Red"].head(1)

,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Intact,IntactValue,SexuponOutcomeValue,ColorValue
ID,,,,,,,,,,,
1,Summer,2015-10-12 12:15:00,Dog,Female,10 months,Labrador Retriever Mix,Red,Yes,1,0,40


In [203]:
train[train.Color=="Red"].head(1)

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Intact,IntactValue,SexuponOutcomeValue,ColorValue
AnimalID,,,,,,,,,,,,,
A671784,Lucy,2014-02-04 17:17:00,Adoption,NaN,Dog,Female,5 months,American Pit Bull Terrier Mix,Red,No,0,0,40


In [204]:
train.head(5).SexuponOutcome.cat.codes

AnimalID
A671945    1
A656520    0
A686464    1
A683430    1
A667013    1
dtype: int8

In [205]:
#train.Breed = train.Breed.str.replace(" Mix", "")
train.Breed = train.Breed.apply(lambda x: x.split("/")[0])
test.Breed = test.Breed.apply(lambda x: x.split("/")[0])

In [206]:
train.Breed = train.Breed.astype("category")
test.Breed = test.Breed.astype("category", categories=train.Breed.cat.categories.values.tolist())

train["BreedValue"] = train.Breed.cat.codes
test["BreedValue"] = test.Breed.cat.codes

## Prediction

In [207]:
#create and train the random forest
    #multi-core CPUs can use: rf = RandomForestClassifier(n_estimators=100, n_jobs=2)
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

d_train = train[["SexuponOutcomeValue", "IntactValue", "ColorValue", "BreedValue"]].values

#d_train
d_target = train.OutcomeType.cat.codes.values

In [208]:
d_train


rf.fit(d_train, d_target)


#d_train.reshape(-1, 1)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
myTest = test.copy()

stopCounter = 100

result = None #DataFrame(columns=["ID", "Adoption", "Died", ])

predictOn = myTest[['SexuponOutcomeValue', 'IntactValue', 'ColorValue', 'BreedValue']]

calcPredictProbas = rf.predict_proba(predictOn.values)
#calcPredictProbas

In [ ]:
ii = 0
for index, row in myTest.iterrows():
    stopCounter = stopCounter - 1
    #if stopCounter < 0: break
    
    probas = calcPredictProbas[ii]
    ii = ii + 1
    #print(probas)
    
    #for i in probas:
    if result is None:
        result = DataFrame(columns=["ID"])
        result.set_index("ID")

        for j in range(0, len(probas)):
            result[train.OutcomeType.cat.categories[j]] = None
            print(train.OutcomeType.cat.categories[j])

    #print(index)

    newLine = [index]
    newLine.extend(probas)
    #print(newLine)
    result.loc[len(result)] = newLine
            
result.head()


Adoption
Died
Euthanasia
Return_to_owner
Transfer


In [ ]:
result["ID"] = result["ID"].astype(int)
result.set_index("ID", inplace=True)
result.to_csv("result.csv")
result